<a href="https://colab.research.google.com/github/HuyenNguyenHelen/Huyen_INFO5731_Spring2020/blob/master/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [1]:

# open and read file

import nltk
nltk.download('punkt')
with open("cleaned_review_content_forAssign3.txt", "r", encoding='utf-8') as file:
  data = file.read()




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
# (1.1) Count the frequency of all the N-grams (N=3)

import pandas
from textblob import TextBlob
tri_grams=TextBlob(data).ngrams(3)

hist=dict()
for gram in tri_grams:
  gram=tuple(gram)
  hist[gram]=hist.get(gram,0)+1

pandas.DataFrame.from_dict(hist, orient = 'index', columns=['tri-grams frequency']).head(10)



,tri-grams frequency
"(person, saw, hype)",1
"(saw, hype, claim)",1
"(hype, claim, masterpiec)",1
"(claim, masterpiec, overreact)",1
"(masterpiec, overreact, overblown)",1
"(overreact, overblown, excit)",1
"(overblown, excit, anoth)",1
"(excit, anoth, joker)",1
"(anoth, joker, base)",1
"(joker, base, film)",1


In [4]:
#(1.2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.
import pandas
monty = TextBlob (data)
bi_grams=TextBlob(data).ngrams(2)
#print(bi_grams)

# make a dict of bigrams and their frequency
hist={}
for gram in bi_grams:
  gram=tuple(gram)
  hist[gram]=hist.get(gram,0)+1
# calculate the probabilities of bigrams
bigr_prob={}
for key, val in hist.items():
  bigr_prob[key]=val/monty.words.count(key[0])
   
df = pandas.DataFrame.from_dict(bigr_prob, orient = 'index', columns=['bigrams probabilities'])
df['bigram frequency']=[val for val in hist.values()]
df.head(10)



,bigrams probabilities,bigram frequency
"(person, saw)",0.052632,1
"(saw, hype)",0.076923,1
"(hype, claim)",0.066667,1
"(claim, masterpiec)",0.500000,1
"(masterpiec, overreact)",0.047619,1
"(overreact, overblown)",1.000000,1
"(overblown, excit)",1.000000,1
"(excit, anoth)",0.500000,1
"(anoth, joker)",0.083333,1
"(joker, base)",0.004762,1


In [8]:
#(1.3) Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets).

# Extract all the noun phrases 
import pandas
import nltk
nltk.download('brown')

monty=TextBlob(data)

# make a dictionary of noun phrases and their frequency in the whole corpus
hist1={}
for np in monty.noun_phrases:
  hist1[np]=hist1.get(np,0)+1
all_nphrases= [val for val in hist1.values()]
# find the maximum frequency of these noun phrases in the whole dataset
max_refreq = max([val for val in hist1.values()])


# calculate the relative probability of reviews to others based on noun phrases
re_prob=[]
for line in data.splitlines():
  monty=TextBlob(line)
  nphrases= monty.noun_phrases
  hist2={}  # make a dictionary of noun phrases and their frequency in each review
  for np in nphrases:
    hist2[np]=hist2.get(np,0)+1
  for key, val in hist2.items():
    hist2[key] = val/max_refreq
  re_prob.append(hist2)

doc_index=[n for n in range(1, len(re_prob)+1)]

df=pandas.DataFrame (re_prob, index=doc_index)
df.fillna(0, inplace=True) # set Null as 0
df





[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


,hype claim masterpiec overreact overblown excit anoth joker base film,bit pretenti trailer,wrong massiv achiev cinema,rare day age cgi nonsens reboot,reboot sort standalon origin tale impecc,finish echo resembl,joker origin,comic past joaquin,sweat cri everi drop magnific dedic,heath ledger,proud undoubtedli,sinc heath joker,brilliant bleak set tone palpabl,film place blown,everi audienc member awestruck wit film,transport charact world,believ hype,rever transcend masterpiec cinema,everi movi,impact joaquin,scenographi brillianc grotesqu haunt cringi,time mesmer wont blink eye,serious funni moment emot rollercoast sometim multipl emot poppingup timethi,typic actionriddl,superhero movi,proper psycholog thrillerdrama singl,alon isol truli relat,sorri charact lot peopl,encourag violenc truli movi encourag everi,u becom,everyon respect,truli masterpiec,hollywood film,film decad truli,comic book chillingli realist,real ife remark,direct cinematographi music act peopl surpris,violent necess messag societi reflect underappreciatedunrecognizedbulli peopl,someth way show class differ,corrupt rich talent rule,...,friend visit school,child care sibl,tall parent,thing futur,chaosbtw good play mr phoenix,night highli disappoint possibl,joker movi ive,imdbspoil belowi,movi interest moment attempt touch,ignor marxist underton,rich evil oppressor,poor good oppress mask,violent protestor,main reason dislik movi joker way geniu,super villain,outwit charact audienc cun intellig sob stori loser way control anyth,life everyon,easili outwit caus fire,sad dude,bulli bunch teenag kid,evil geniu,eventu medic,pure luck,rich men,fame complet coincid complet controlthi joker chanc fight batman time bruce,becom batman joker,thing pathet joker fight ptsd,movi rip movi doesnt redeem valu,impli bruce wayn father joker kind joke dc movi origin stori work hack,idea scorses tri,man kind folk hero unfortun joaquim phoenix overact,liter everi singl scene movi becom tiresom,joaquin phoenix asid overal movi way overhyp way,slow repetit,setback havent,dramat dire,justifi eventu behavior laugh viciou importantli place sequenc,final clear abl mobil anarchist entir gotham,dont fall movi total overhyp border bore irrit joaquin good movi stori goe aimlessli joaquin behavior crazi laughter repetiti,star review mayb famili friend peopl work movi
1,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [9]:
# (2.1) build the documents-terms weights (tf*idf) matrix bold text.
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer

doc=[doc for doc in data.splitlines()]
corpus = {v: k for v, k in enumerate(data.splitlines(),1)}
tfidf=TfidfVectorizer(input=doc, max_features=50, min_df=0.2, lowercase=True, analyzer='word', stop_words='english', ngram_range=(1,3))
tfs=tfidf.fit_transform(corpus.values())
feature_names= tfidf.get_feature_names()
corpus_index=[n for n in corpus]

df=pandas.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
df



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111
act,0.192691,0.000000,0.000000,0.115510,0.000000,0.000000,0.000000,0.000000,0.131581,0.528928,0.119421,0.000000,0.000000,0.697005,0.000000,0.443831,0.184646,0.000000,0.143778,0.143665,0.000000,0.286503,0.149785,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.131079,0.000000,0.000000,0.0,0.000000,0.518869,0.000000,0.000000,...,0.304027,0.431570,0.000000,0.0,0.542830,0.169233,0.000000,0.000000,0.000000,0.000000,0.574229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.824535,0.000000,0.000000,0.000000,0.082804,0.0,0.361305,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058990,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
best,0.452600,0.321787,0.000000,0.406970,0.000000,0.000000,0.627677,0.000000,0.154531,0.310592,0.000000,0.000000,0.249172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.113876,0.000000,0.000000,0.228605,0.246725,0.432970,0.373517,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.609370,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.212504,0.855945,0.000000,0.000000,0.000000,0.568210,0.890263,0.000000,0.000000,0.098095,0.000000,0.369751,0.419189,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
charact,0.180796,0.257083,0.192919,0.000000,0.000000,0.000000,0.000000,0.000000,0.123458,0.000000,0.560244,0.000000,0.000000,0.000000,0.000000,0.000000,0.519743,0.000000,0.134902,0.269593,0.272935,0.268817,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050653,0.000000,0.319837,0.0,0.368963,0.385804,0.000000,0.0,0.000000,0.000000,0.000000,0.265746,...,0.570518,0.000000,0.135930,0.0,0.000000,0.158787,0.000000,0.000000,0.169774,0.000000,0.000000,0.434085,0.000000,0.000000,0.000000,0.000000,0.000000,0.078370,0.000000,0.098467,0.000000,0.244280,0.233077,0.0,0.000000,0.000000,0.000000,0.292153,0.178382,0.000000,0.000000,0.000000,0.000000,0.110697,0.181726,0.170037,0.000000,0.000000,0.000000,0.0
dark,0.000000,0.000000,0.000000,0.000000,0.547030,0.577790,0.000000,0.000000,0.149662,0.300806,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.361749,0.000000,0.061404,0.000000,0.000000,0.0,0.149091,0.000000,0.000000,0.0,0.378501,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.192489,0.000000,0.000000,0.000000,0.000000,0.000000,0.263110,0.000000,0.000000,0.000000,0.218313,0.000000,0.000000,0.000000,0.358101,0.202991,0.098709,0.282548,0.0,0.000000,0.000000,0.000000,0.000000,0.216244,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
dont,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.213367,0.000000,0.000000,0.000000,0.000000,0.000000,0.346167,0.000000,0.000000,0.000000,0.367515,0.485748,0.062383,0.593898,0.393902,0.0,0.151468,0.000000,0.363062,0.0,0.384534,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.167408,0.0,0.000000,0.000000,0.000000,0.597171,0.000000,0.000000,0.000000,0.267304,0.000000,0.000000,0.000000,0.000000,0.693149,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.407738,0.143923,0.439382,0.000000,0.000000,0.000000,0.000000,0.136331,0.447617,0.000000,0.000000,0.000000,0.350003,0.0
feel,0.000000,0.000000,0.712783,0.000000,0.000000,0.000000,0.000000,0.000000,0.152048,0.000000,0.275993,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166142,0.332023,0.112047,0.000000,0.173084,0.449864,0.000000,

In [10]:
#(2.2) rank the documents with respect to query 

# design a query 
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
from textblob import Word
query="Is the Joker the best movie of the year?"

# preprocess query (lowercase, remove punctuations, remove stopwords, stem and lematize)
query=query.lower()
query=re.sub('[^\w\s]', '', query)
stop=stopwords.words('english')
query= " ".join(x for x in query.split() if x not in stop)
st=PorterStemmer()
query=" ".join([st.stem(word) for word in query.split()])
query= " ".join([Word(word).lemmatize() for word in query.split()])
print(query)

# append query to the working corpus (all cleaned reviews)
with open("reviews_query_forAssign3.txt", "a", encoding='utf-8') as file:
  file.write(data)
  file.write(query)
with open("reviews_query_forAssign3.txt", "r", encoding='utf-8') as file:
  data_query = file.read()






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
joker best movi year


In [11]:
# Caculate tfidf, and extract features representing all reviews and the query

doc=[doc for doc in data_query.splitlines()]
corpus = {v: k for v, k in enumerate(data_query.splitlines(),1)}
tfidf=TfidfVectorizer(input=doc, max_features=50, min_df=0.2, lowercase=True, analyzer='word', stop_words='english', ngram_range=(1,1))
tfs=tfidf.fit_transform(corpus.values())
feature_names= tfidf.get_feature_names()
corpus_index=[n for n in corpus]

df=pandas.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
df



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112
act,0.193588,0.000000,0.000000,0.115836,0.000000,0.000000,0.000000,0.000000,0.133171,0.530635,0.120730,0.000000,0.000000,0.697601,0.000000,0.497159,0.188209,0.000000,0.144573,0.145575,0.000000,0.286652,0.149815,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.131454,0.000000,0.000000,0.0,0.000000,0.522396,0.000000,0.000000,...,0.480465,0.000000,0.0,0.543230,0.171804,0.000000,0.000000,0.000000,0.000000,0.574261,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.826589,0.000000,0.000000,0.000000,0.084161,0.0,0.361974,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.059296,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
best,0.447154,0.317286,0.000000,0.401342,0.000000,0.000000,0.622265,0.000000,0.153800,0.306418,0.000000,0.000000,0.246335,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.114501,0.000000,0.000000,0.225113,0.268712,0.428185,0.369048,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.603321,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.209667,0.852811,0.000000,0.000000,0.000000,0.646171,0.888353,0.000000,0.000000,0.096880,0.000000,0.364754,0.444249,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.786957
charact,0.181687,0.257838,0.193221,0.000000,0.000000,0.000000,0.000000,0.000000,0.124984,0.000000,0.566538,0.000000,0.000000,0.000000,0.000000,0.000000,0.529915,0.000000,0.135685,0.273250,0.279142,0.269029,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050894,0.000000,0.320914,0.0,0.370118,0.386853,0.000000,0.0,0.000000,0.000000,0.000000,0.266169,...,0.000000,0.136482,0.0,0.000000,0.161242,0.000000,0.000000,0.170383,0.000000,0.000000,0.448031,0.000000,0.000000,0.000000,0.000000,0.000000,0.078729,0.000000,0.098804,0.000000,0.245048,0.236961,0.0,0.000000,0.000000,0.000000,0.292825,0.182396,0.000000,0.000000,0.000000,0.000000,0.111301,0.182239,0.170955,0.000000,0.000000,0.000000,0.0,0.000000
dark,0.000000,0.000000,0.000000,0.000000,0.547654,0.597903,0.000000,0.000000,0.151396,0.301627,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.363278,0.000000,0.061649,0.000000,0.000000,0.0,0.149444,0.000000,0.000000,0.0,0.378521,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.195317,0.000000,0.000000,0.000000,0.000000,0.000000,0.271355,0.000000,0.000000,0.000000,0.219191,0.000000,0.000000,0.000000,0.359051,0.218652,0.098944,0.287036,0.0,0.000000,0.000000,0.000000,0.000000,0.220941,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
dont,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.217365,0.000000,0.000000,0.000000,0.000000,0.000000,0.346046,0.000000,0.000000,0.000000,0.369048,0.486734,0.062628,0.594081,0.394905,0.0,0.151818,0.000000,0.363082,0.0,0.384534,0.000000,0.000000,0.000000,...,0.000000,0.167949,0.0,0.000000,0.000000,0.000000,0.597100,0.000000,0.000000,0.000000,0.275665,0.000000,0.000000,0.000000,0.000000,0.695281,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.408665,0.144136,0.448900,0.000000,0.000000,0.000000,0.000000,0.136963,0.448514,0.000000,0.000000,0.000000,0.350471,0.0,0.000000
feel,0.000000,0.000000,0.713312,0.000000,0.000000,0.000000,0.000000,0.000000,0.153800,0.000000,0.278864,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166969,0.336252,0.114501,0.000000,0.173023,0.450226,0.000000

In [13]:
# Calculate cosine similarity of the query to others documents by using SKLEARN
import pandas
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# looping over documents to calculate cosine similarity of query (doc111) to others
# make a dict of doc and results
i=1
quer_sim = {}
while i <=111:
  cosim=cosine_similarity(tfs[111], tfs[i])  # a function of sklearn
  quer_sim['doc'+str(i)] = cosim
  i+=1
print(quer_sim)
sorted_quer_sim = sorted (quer_sim, key=quer_sim.get, reverse=True)
# sort the results to rank the similarity of each doc to the query
quer_sim_rank = {key:rank for rank, key in enumerate(sorted (quer_sim, key=quer_sim.get, reverse=True),1)}
cosim_list = [quer_sim[doc]for doc in sorted_quer_sim]

rankdf=pandas.DataFrame.from_dict (quer_sim_rank,orient = 'index', columns = ['ranking'])
rankdf ['cosine similarity'] = cosim_list
rankdf.head(20)


{'doc1': array([[0.38310131]]), 'doc2': array([[0.14996536]]), 'doc3': array([[0.35243094]]), 'doc4': array([[0.26914091]]), 'doc5': array([[0.08306851]]), 'doc6': array([[0.68593216]]), 'doc7': array([[0.17515604]]), 'doc8': array([[0.49304663]]), 'doc9': array([[0.51821196]]), 'doc10': array([[0.43402395]]), 'doc11': array([[0.17279545]]), 'doc12': array([[0.65493295]]), 'doc13': array([[0.16938191]]), 'doc14': array([[0.08747175]]), 'doc15': array([[0.12071319]]), 'doc16': array([[0.18279329]]), 'doc17': array([[0.33981804]]), 'doc18': array([[0.53030804]]), 'doc19': array([[0.35003709]]), 'doc20': array([[0.40562549]]), 'doc21': array([[0.06960087]]), 'doc22': array([[0.04732552]]), 'doc23': array([[0.27180891]]), 'doc24': array([[0.47144984]]), 'doc25': array([[0.66121959]]), 'doc26': array([[0.52318916]]), 'doc27': array([[0.33779275]]), 'doc28': array([[0.37800895]]), 'doc29': array([[0.1248984]]), 'doc30': array([[0.35708702]]), 'doc31': array([[0.]]), 'doc32': array([[0.376824

,ranking,cosine similarity
doc111,1,[[1.0]]
doc85,2,[[0.8858611717264935]]
doc80,3,[[0.8774040902430584]]
doc68,4,[[0.8508540078892142]]
doc6,5,[[0.6859321622120038]]
doc84,6,[[0.6852504609363683]]
doc63,7,[[0.6617040911219179]]
doc25,8,[[0.6612195933736404]]
doc12,9,[[0.6549329533616601]]
doc91,10,[[0.6252709242868153]]


In [15]:
# Calculate cosine similarity of query with documents manually 

import math
import numpy as np

# define a function to calculate cosine similarity of two vectors
def cosim (v1,v2):
  x=v1.toarray()
  y=v2.toarray()
  return np.sum(x*y)/math.sqrt(np.sum(x*x)*np.sum(y*y))

# Using defined function to calculate cosine similarity of the query to each doc 
# looping through each doc, calculate and store in a dict
i=1
query_sim = {}
while i <=111:
  cosin_similarity=cosim(tfs[111], tfs[i])
  cosin_similarity = str(cosin_similarity)
  query_sim['doc'+str(i)] = cosin_similarity
  i+=1
query_sim_noNul= {k:float(v) for k,v in query_sim.items() if v != 'nan'}  # remove Null values from the dict
# sort the results to rank
sorted_query_sim = sorted (query_sim_noNul, key=query_sim_noNul.get, reverse=True)
query_sim_rank = {key:rank for rank, key in enumerate(sorted_query_sim,1)}

cosim_list = [query_sim_noNul[doc] for doc in sorted_query_sim]
rank_df=pandas.DataFrame.from_dict (query_sim_rank,orient = 'index', columns = ['ranking'])
rank_df ['cosine similarity'] = cosim_list
rank_df.head(20)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


,ranking,cosine similarity
doc111,1,1.000000
doc85,2,0.885861
doc80,3,0.877404
doc68,4,0.850854
doc6,5,0.685932
doc84,6,0.685250
doc63,7,0.661704
doc25,8,0.661220
doc12,9,0.654933
doc91,10,0.625271


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

# Link: 
https://github.com/HuyenNguyenHelen/Huyen_INFO5731_Spring2020/blob/master/Huyen_labeled_review_sentiments.csv 
